In [732]:
import pandas as pd
import numpy as np
import seaborn as sn
from numbers import Number
from time import time
from ta import momentum as mo
from ta import volume as vo
from ta import trend as tr
import sklearn.ensemble as ens

In [733]:
year = 31536000
period1 = round(time() - (year * 10))
period2 = round(time())
asset = 'aapl'
    
hist_data = pd.read_csv(f'https://query1.finance.yahoo.com/v7/finance/download/{asset}?period1={period1}&period2={period2}&interval=1d&events=history&includeAdjustedClose=true')

In [734]:
# ROC de Y em relação a X
def change(x, y):
    if len(x) == len(y):
        change_arr = []
        for i in range(0, len(x)):
            if isinstance(x[i], Number) and isinstance(y[i], Number):
                change_arr.append(((y[i] - x[i]) / x[i]) * 100)
            else:
                change_arr.append('NaN')
        
        return change_arr
    else:
        print(f"Error: x lenght {len(x)}, y lenght {len(y)}")

In [735]:
# ROC de Y em relação a X
def change(x, y):
    if len(x) == len(y):
        change_arr = []
        for i in range(0, len(x)):
            if isinstance(x[i], Number) and isinstance(y[i], Number):
                change_arr.append(((y[i] - x[i]) / x[i]) * 100)
            else:
                change_arr.append('NaN')
        
        return change_arr
    else:
        print(f"Error: x lenght {len(x)}, y lenght {len(y)}")

In [736]:
# print(change([1, 2],[2, 1])) # [100.0, -50.0] Yn > Xn se valor positivo
trend_window = 3
slice_start = 50
slice_plus = 0

roc = mo.roc(hist_data['Close'], window=1)
rsi = mo.rsi(hist_data['Close'])
stochrsi = mo.stochrsi(hist_data['Close'])
stochrsi_d = mo.stochrsi_d(hist_data['Close'])
stochrsi_k = mo.stochrsi_k(hist_data['Close'])

tsi = mo.tsi(hist_data['Close'])
tsi_signal = tr.ema_indicator(tsi, window=13)

macd = tr.macd(hist_data['Close'])
macd_diff = tr.macd_diff(hist_data['Close'])
macd_signal = tr.macd_signal(hist_data['Close'])

stoch = mo.stoch(hist_data['High'], hist_data['Low'], hist_data['Close'])
ema_w8 = tr.ema_indicator(hist_data['Close'], window=8)
ema_w20 = tr.ema_indicator(hist_data['Close'], window=20)

sma_w13 = tr.sma_indicator(hist_data['Close'], window=13)
sma_w25 = tr.sma_indicator(hist_data['Close'], window=25)

roc_close_ema_w8 = change(ema_w8, hist_data['Close'])
roc_ema_w8_ema_w20 = change(ema_w20, ema_w8)
roc_close_ema_w20 = change(ema_w20, hist_data['Close'])
roc_close_open = change(hist_data['Open'], hist_data['Close'])

roc_close_sma_w13 = change(sma_w13, hist_data['Close'])
roc_sma_w13_sma_w25 = change(sma_w25, sma_w13)
roc_close_sma_w25 = change(sma_w25, hist_data['Close'])

trend = mo.roc(hist_data['Close'], window=trend_window).map(lambda n : 'HIGH' if n > 0 else 'LOW')

df = pd.DataFrame({
    'close': np.array(list(hist_data['Close'][slice_start:-(trend_window + slice_plus)])),
#     'roc': np.array(list(roc[slice_start:-(trend_window + slice_plus)])),
    'rsi': np.array(list(rsi[slice_start:-(trend_window + slice_plus)])),
#     'stoch': np.array(list(stoch[slice_start:-(trend_window + slice_plus)])),
#     'stochrsi': np.array(list(stochrsi[slice_start:-(trend_window + slice_plus)])),
#     'stochrsi_d': np.array(list(stochrsi_d[slice_start:-(trend_window + slice_plus)])),
#     'stochrsi_k': np.array(list(stochrsi_k[slice_start:-(trend_window + slice_plus)])),
    'macd': np.array(list(macd[slice_start:-(trend_window + slice_plus)])),
    'macd_diff': np.array(list(macd_diff[slice_start:-(trend_window + slice_plus)])),
    'macd_signal': np.array(list(macd_signal[slice_start:-(trend_window + slice_plus)])),
    'tsi': np.array(list(tsi[slice_start:-(trend_window + slice_plus)])),
    'tsi_signal': np.array(list(tsi_signal[slice_start:-(trend_window + slice_plus)])),
#     'ema_w8': np.array(list(ema_w8[slice_start:-(trend_window + slice_plus)])),
#     'ema_w20': np.array(list(ema_w20[slice_start:-(trend_window + slice_plus)])),
    
#     'roc_close_ema_w8': np.array(list(roc_close_ema_w8[slice_start:-(trend_window + slice_plus)])),
#     'roc_ema_w8_ema_w20': np.array(list(roc_ema_w8_ema_w20[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_ema_w20': np.array(list(roc_close_ema_w20[slice_start:-(trend_window + slice_plus)])),
    
#     'roc_close_sma_w13': np.array(list(roc_close_sma_w13[slice_start:-(trend_window + slice_plus)])),
#     'roc_sma_w13_sma_w25': np.array(list(roc_sma_w13_sma_w25[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_sma_w25': np.array(list(roc_close_sma_w25[slice_start:-(trend_window + slice_plus)])),
    'trend': np.array(list(trend[(slice_start + trend_window)::]))
})


df

,close,rsi,macd,macd_diff,macd_signal,tsi,tsi_signal,ema_w8,ema_w20,trend
0,103.650002,63.610703,2.134708,0.200846,1.933862,22.313187,16.618128,102.960823,100.879689,HIGH
1,102.589996,58.959110,2.004735,0.056698,1.948037,21.646379,17.336450,102.878417,101.042576,HIGH
2,102.580002,58.915366,1.879261,-0.055020,1.934281,21.074880,17.870511,102.812103,101.188997,HIGH
3,99.900002,48.519487,1.545750,-0.310825,1.856575,17.570613,17.827669,102.164969,101.066236,HIGH
4,100.480003,50.553019,1.313104,-0.434776,1.747881,15.170018,17.448004,101.790532,101.010404,HIGH
...,...,...,...,...,...,...,...,...,...,...
61,NaN,56.671012,1.010490,0.045687,0.964803,13.657170,11.855701,102.775472,101.499385,LOW
62,NaN,56.671012,1.010490,0.036549,0.973941,13.675412,12.115660,102.775472,101.499385,LOW
63,NaN,56.671012,1.010490,0.029239,0.981251,13.690992,12.340707,102.775472,101.499385,LOW
64,NaN,56.671012,1.010490,0.023392,0.987098,13.704305,12.535507,102.775472,101.499385,LOW


In [737]:
train = df.iloc[0:-5,0::].sample(frac=1)
test = df.iloc[-5::,0::]

X_train = train.iloc[:,0:-1]
y_train = train.iloc[:,-1]

X_test = test.iloc[:,0:-1]
y_test = test.iloc[:,-1].values

rf = ens.AdaBoostClassifier(n_estimators=1000, learning_rate=1.0)
rf_model = rf.fit(X_train, y_train)
pred = rf_model.predict(X_test)

corr_pred = 0
wrong_pred = 0
for i in list(range(0, len(pred))):
    if pred[i] == 'LOW' and y_test[i] == 'LOW':
        corr_pred = corr_pred + 1
    elif pred[i] == 'HIGH' and y_test[i] == 'HIGH':
        corr_pred = corr_pred + 1
    else:
        wrong_pred = wrong_pred + 1

print(f"Corret Prediction: {corr_pred}, {(corr_pred / len(pred)) * 100}%")
print(f"Wrong Prediction: {wrong_pred}, {(wrong_pred / len(pred)) * 100}%")
print(pred)
print(y_test)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
X_train.fillna(X_train.mean())

,close,rsi,macd,macd_diff,macd_signal,tsi,tsi_signal,ema_w8,ema_w20
356,125.910004,40.431877,-0.167088,-1.040810,0.873723,-0.991884,9.579885,128.916286,129.994647
5,70.102501,72.732011,1.448592,0.180220,1.268372,33.964941,34.242926,68.469146,67.073559
239,118.639999,53.763604,0.988981,0.341598,0.647383,6.537134,3.136850,118.408912,117.243418
68,61.195000,38.757939,-3.875245,-0.946465,-2.928779,-20.125050,-15.016791,64.401522,68.701646
55,74.702499,46.523660,-1.574386,-1.181375,-0.393011,-16.476995,3.330920,73.530395,76.030646
...,...,...,...,...,...,...,...,...,...
394,142.020004,78.416148,3.106833,0.941430,2.165403,36.436014,21.097570,137.403051,133.804011
300,129.869995,41.791694,-0.154499,-1.090440,0.935941,-0.843261,8.239389,132.458079,133.627836
427,148.190002,56.505400,1.531282,-0.343307,1.874590,17.367158,21.952627,147.872967,146.913059
179,126.522499,79.387958,7.024151,0.833279,6.190872,51.436748,44.598759,121.585008,114.716513


In [ ]:
X_train

,close,rsi,macd,macd_diff,macd_signal,tsi,tsi_signal,ema_w8,ema_w20
356,125.910004,40.431877,-0.167088,-1.040810,0.873723,-0.991884,9.579885,128.916286,129.994647
5,70.102501,72.732011,1.448592,0.180220,1.268372,33.964941,34.242926,68.469146,67.073559
239,118.639999,53.763604,0.988981,0.341598,0.647383,6.537134,3.136850,118.408912,117.243418
68,61.195000,38.757939,-3.875245,-0.946465,-2.928779,-20.125050,-15.016791,64.401522,68.701646
55,74.702499,46.523660,-1.574386,-1.181375,-0.393011,-16.476995,3.330920,73.530395,76.030646
...,...,...,...,...,...,...,...,...,...
394,142.020004,78.416148,3.106833,0.941430,2.165403,36.436014,21.097570,137.403051,133.804011
300,129.869995,41.791694,-0.154499,-1.090440,0.935941,-0.843261,8.239389,132.458079,133.627836
427,148.190002,56.505400,1.531282,-0.343307,1.874590,17.367158,21.952627,147.872967,146.913059
179,126.522499,79.387958,7.024151,0.833279,6.190872,51.436748,44.598759,121.585008,114.716513


In [744]:

# X_test.iloc[-1,0] = np.nan
X_train[X_train==np.inf]=np.nan
X_train.fillna(X_train.mean(), inplace=True)
X_train.iloc[-1,:].to_frame(0).T

,close,rsi,macd,macd_diff,macd_signal,tsi,tsi_signal,ema_w8,ema_w20
0,100.776539,56.671012,1.01049,0.21785,0.79264,13.402044,8.487047,102.775472,101.499385
